In [1]:
import mysql.connector as sql
import pandas as pd
import datetime as dt
import numpy as np
from sorted_months_weekdays import *
from sort_dataframeby_monthorweek import *
import matplotlib.pyplot as plt
import math
import json
from pandas.io.json import json_normalize

In [2]:
db_connection = sql.connect(host='localhost', database='collection_visits', user='pandas', password='password')

visits = pd.read_sql('SELECT * FROM Visits', con=db_connection)
visits['created'] = pd.to_datetime(visits['created'], unit='s').dt.tz_localize('utc').dt.tz_convert('US/Eastern')
visits['lastmodified'] = pd.to_datetime(visits['lastmodified'], unit='s').dt.tz_localize('utc').dt.tz_convert('US/Eastern')
visits['registered'] = visits['registered'].replace(0, np.nan)
visits['registered'] = pd.to_datetime(visits['registered'], unit='s').dt.tz_localize('utc').dt.tz_convert('US/Eastern')
visits['deleted'] = visits['deleted'].replace(0, np.nan)
visits['deleted'] = pd.to_datetime(visits['deleted'], unit='s').dt.tz_localize('utc').dt.tz_convert('US/Eastern')
visits['paired'] = visits['paired'].replace(0, np.nan)
visits['paired'] = pd.to_datetime(visits['paired'], unit='s').dt.tz_localize('utc').dt.tz_convert('US/Eastern')
visits['created_ip'].replace(0, np.nan, inplace=True)
visits['registered_ip'].replace(0, np.nan, inplace=True)
visits['paired_ip'].replace(0, np.nan, inplace=True)
# visits['processed'].replace(0, np.nan, inplace=True)
visits['exported'].replace(0, np.nan, inplace=True)
visits['title'].fillna(value=np.nan, inplace=True)
visits['note'].replace(r'^\s*$', np.NaN, regex=True, inplace=True)

In [18]:
visit_opening_hours = visits[['created', 'user_id']]
visit_opening_hours.set_index(['created'], inplace=True)
visit_opening_hours = visit_opening_hours.between_time('10:00', '18:00')
visit_opening_hours

,user_id
created,
2014-10-29 17:10:47-04:00,0
2014-11-03 13:31:58-05:00,0
2014-11-03 16:13:10-05:00,247
2014-11-03 16:16:30-05:00,0
2014-11-03 16:17:36-05:00,0
...,...
2020-07-01 13:55:07-04:00,0
2020-07-02 13:15:41-04:00,28814
2020-07-02 13:15:50-04:00,28814


In [6]:
visits['day_of_week'] = visits.created.dt.dayofweek

In [11]:
saturday = visits[visits['day_of_week']==5]

In [15]:
saturday.set_index('created', inplace=True)

In [16]:
saturday.between_time('10:00', '18:00')

,id,shortcode,user_id,lastmodified,registered,expires,paired,ticket,deleted,created_ip,registered_ip,paired_ip,processed,purchased_by_user_id,note,exported,first_accessed,ignore_first_accessed,title,day_of_week
created,,,,,,,,,,,,,,,,,,,,
2015-03-07 12:53:20-05:00,2713403,dpy6,247,2016-02-23 17:57:43-05:00,2015-03-07 12:53:21-05:00,0,2015-03-07 12:53:34-05:00,"{""status"":""Attendance recorded for ticket: 574...",NaT,1.223001e+09,1.223001e+09,1.223001e+09,1425776826,0,NaN,NaN,NaN,1.0,NaN,5
2015-03-07 16:12:44-05:00,2749023,npqg,0,2016-02-23 17:57:43-05:00,2015-03-07 16:12:45-05:00,0,NaT,"{""status"":""Attendance recorded for ticket: 579...",NaT,9.173936e+08,9.173936e+08,NaN,0,0,NaN,NaN,NaN,1.0,NaN,5
2015-03-14 10:02:11-04:00,3194927,2bb7,0,2017-03-07 05:42:51-05:00,2015-03-14 10:02:11-04:00,0,NaT,"{""status"":""Attendance recorded for ticket: 615...",NaT,6.458940e+08,6.458940e+08,NaN,0,0,NaN,NaN,1.488883e+09,1.0,NaN,5
2015-03-14 10:02:33-04:00,3194929,lstq,0,2016-02-23 17:58:02-05:00,2015-03-14 10:02:32-04:00,0,NaT,"{""status"":""Attendance recorded for ticket: 615...",NaT,6.458940e+08,6.458940e+08,NaN,0,0,NaN,NaN,NaN,1.0,NaN,5
2015-03-14 10:02:33-04:00,3194931,c7kq,0,2016-02-23 17:58:02-05:00,2015-03-14 10:02:33-04:00,0,NaT,"{""status"":""Attendance recorded for ticket: 615...",NaT,6.458940e+08,6.458940e+08,NaN,0,0,NaN,NaN,NaN,1.0,NaN,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-02-29 16:37:03-05:00,132188613,qqz4q,0,2020-02-29 16:37:05-05:00,2020-02-29 16:37:05-05:00,0,NaT,"{""status"":""Attendance recorded for ticket: 937...",NaT,2.691693e+09,2.691693e+09,NaN,0,0,NaN,NaN,NaN,NaN,NaN,5
2020-02-29 16:37:30-05:00,132188717,24hqm,0,2020-02-29 17:28:33-05:00,2020-02-29 16:37:31-05:00,0,NaT,"{""status"":""Attendance recorded for ticket: 937...",NaT,2.691693e+09,2.691693e+09,NaN,0,0,NaN,NaN,1.583015e+09,NaN,NaN,5
2020-02-29 16:53:09-05:00,132188911,5qvc,0,2020-02-29 16:53:10-05:00,2020-02-29 16:53:10-05:00,0,NaT,"{""status"":""Attendance recorded for ticket: 937...",NaT,2.691693e+09,2.691693e+09,NaN,0,0,NaN,NaN,NaN,NaN,NaN,5


In [19]:
saturday.between_time('18:00', '21:00')
#40712 more visits in the last 3 hours. 6% of the total visits. 

,id,shortcode,user_id,lastmodified,registered,expires,paired,ticket,deleted,created_ip,registered_ip,paired_ip,processed,purchased_by_user_id,note,exported,first_accessed,ignore_first_accessed,title,day_of_week
created,,,,,,,,,,,,,,,,,,,,
2015-04-25 18:34:53-04:00,4560421,2yyn,0,2017-03-07 11:40:20-05:00,2015-04-25 18:34:54-04:00,0,NaT,"{""status"":""Attendance recorded for ticket: 906...",NaT,6.458940e+08,6.458940e+08,NaN,0,0,NaN,NaN,1.488905e+09,1.0,NaN,5
2015-04-25 18:35:14-04:00,4560427,5r6c,0,2017-03-07 16:28:00-05:00,2015-04-25 18:35:15-04:00,0,NaT,"{""status"":""Attendance recorded for ticket: 906...",NaT,6.458940e+08,6.458940e+08,NaN,0,0,NaN,NaN,1.488922e+09,1.0,NaN,5
2015-05-16 18:13:30-04:00,5117117,zhyq,2274,2016-02-23 18:04:56-05:00,2015-05-16 18:13:31-04:00,0,2015-05-17 15:48:07-04:00,"{""status"":""Attendance recorded for ticket: 101...",NaT,6.458940e+08,6.458940e+08,1.203178e+09,1431892093,0,NaN,NaN,NaN,1.0,NaN,5
2015-05-16 19:54:27-04:00,5119085,bqb6,0,2016-02-23 18:04:56-05:00,2015-05-16 19:54:28-04:00,0,NaT,"{""status"":""Attendance recorded for ticket: 102...",NaT,6.458940e+08,6.458940e+08,NaN,0,0,NaN,NaN,NaN,1.0,NaN,5
2015-06-20 19:31:47-04:00,85571705,hf6d,3067,2016-02-23 18:07:43-05:00,2015-06-20 19:31:48-04:00,0,2015-06-20 19:40:06-04:00,"{""status"":""Attendance recorded for ticket: 117...",NaT,6.458940e+08,6.458940e+08,2.691694e+09,1434843607,0,NaN,NaN,NaN,1.0,NaN,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-12-28 19:43:52-05:00,131222583,hpty3,0,2019-12-28 19:43:53-05:00,2019-12-28 19:43:53-05:00,0,NaT,"{""status"":""Attendance recorded for ticket: 916...",NaT,2.691693e+09,2.691693e+09,NaN,0,0,NaN,NaN,NaN,NaN,NaN,5
2019-12-28 19:44:00-05:00,131222587,5brxp,0,2019-12-28 19:44:01-05:00,2019-12-28 19:44:01-05:00,0,NaT,"{""status"":""Attendance recorded for ticket: 916...",NaT,2.691693e+09,2.691693e+09,NaN,0,0,NaN,NaN,NaN,NaN,NaN,5
2019-12-28 19:48:13-05:00,131222915,5zg3g,0,2019-12-28 19:48:15-05:00,2019-12-28 19:48:15-05:00,0,NaT,"{""status"":""Attendance recorded for ticket: 916...",NaT,2.691693e+09,2.691693e+09,NaN,0,0,NaN,NaN,NaN,NaN,NaN,5


In [20]:
tuesday = visits[visits['day_of_week']==1]

In [24]:
tuesday.set_index('created', inplace=True)

In [25]:
tuesday.between_time('10:00', '18:00')

,id,shortcode,user_id,lastmodified,registered,expires,paired,ticket,deleted,created_ip,registered_ip,paired_ip,processed,purchased_by_user_id,note,exported,first_accessed,ignore_first_accessed,title,day_of_week
created,,,,,,,,,,,,,,,,,,,,
2014-11-04 10:20:54-05:00,797007,4444555544448,0,2016-02-23 17:57:38-05:00,2014-11-04 10:20:55-05:00,0,NaT,"{""tessitura_spoofing"":1,""ticket_id"":""797007"",""...",NaT,NaN,NaN,NaN,0,0,NaN,NaN,NaN,1.0,NaN,1
2014-11-04 15:32:36-05:00,797837,2014114u142,142,2016-02-23 17:57:38-05:00,2014-11-04 15:32:37-05:00,0,2014-11-04 15:36:04-05:00,"{""tessitura_spoofing"":1,""ticket_id"":""797837"",""...",NaT,NaN,NaN,NaN,0,0,NaN,NaN,NaN,1.0,NaN,1
2014-11-04 15:36:20-05:00,797915,2014114u247,0,2016-02-23 17:57:38-05:00,2014-11-04 15:36:20-05:00,0,NaT,"{""tessitura_spoofing"":1,""ticket_id"":""797915"",""...",NaT,NaN,NaN,NaN,0,0,NaN,NaN,NaN,1.0,NaN,1
2014-11-18 11:16:28-05:00,1049573,20141118u3,0,2016-02-23 17:57:39-05:00,2014-11-18 11:16:28-05:00,0,NaT,"{""tessitura_spoofing"":1,""ticket_id"":""1049573"",...",NaT,NaN,NaN,NaN,0,0,NaN,NaN,NaN,1.0,NaN,1
2014-11-18 11:45:51-05:00,1049597,1tc0,0,2016-02-23 17:57:39-05:00,2014-11-18 11:45:52-05:00,0,NaT,"{""tessitura_spoofing"":1,""ticket_id"":""1049597"",...",NaT,NaN,NaN,NaN,0,0,NaN,NaN,NaN,1.0,NaN,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-03-03 17:37:29-05:00,132229077,3mbd3,0,2020-03-03 17:37:30-05:00,2020-03-03 17:37:30-05:00,0,NaT,"{""status"":""Attendance recorded for ticket: 938...",NaT,2.691693e+09,2.691693e+09,NaN,0,0,NaN,NaN,NaN,NaN,NaN,1
2020-03-03 17:38:09-05:00,132229089,p3q2,0,2020-03-04 13:23:59-05:00,2020-03-03 17:38:10-05:00,0,NaT,"{""status"":""Attendance recorded for ticket: 938...",NaT,2.691693e+09,2.691693e+09,NaN,0,0,NaN,NaN,1.583346e+09,NaN,NaN,1
2020-03-03 17:49:22-05:00,132229455,6y4f,0,2020-03-03 19:57:10-05:00,2020-03-03 17:49:24-05:00,0,NaT,"{""status"":""Attendance recorded for ticket: 938...",NaT,2.691693e+09,2.691693e+09,NaN,0,0,NaN,NaN,1.583283e+09,NaN,NaN,1
